In [ ]:
import stats_functions as sf
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow
import emission.core.get_database as edb
from emission.core.wrapper.user import User
import matplotlib.pyplot as plt

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get('2018-03-28', 'YYYY-MM-DD').timestamp, arrow.get('2018-06-06', 'YYYY-MM-DD').timestamp)
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)
client_nav_events_df.head()

In [ ]:
expanded_trip_events = client_nav_events_df[client_nav_events_df.name == "expanded_trip"]

In [ ]:
information = []
emotion = []
control = []
information_uuids = set()
emotion_uuids = set()
control_uuids = set()
no_clients = set()
for i in range(len(expanded_trip_events)):
    uuid = expanded_trip_events[i]['user_id']
    try:
        client = edb.get_profile_db().find_one({"user_id": uuid})['client']
        if client == 'urap-2017-information':
            information.append(expanded_trip_events[i])
            information_uuids.add(uuid)
        elif client == 'urap-2017-emotion':
            emotion.append(expanded_trip_events[i])
            emotion_uuids.add(uuid)
        elif client == 'urap-2017-control':
            control.append(expanded_trip_events[i])
            control_uuids.add(uuid)
    except:
        no_clients.add(str(uuid))
for elem in no_clients:
    print(elem + " doesnt have a client for some reason")

print(str(len(expanded_trip_events)) + " events")

In [ ]:
emotion_num_users = len(emotion_uuids)
information_num_users = len(information_uuids)
control_num_users = len(control_uuids)
emotion_total_expanded_trips = len(emotion)
information_total_expanded_trips = len(information)
control_total_expanded_trips = len(control)
print(emotion_num_users, information_num_users, control_num_users)

In [ ]:
objects = ('Emotion', 'Information', 'Control')
y_pos = range(len(objects))
performance = [emotion_total_expanded_trips, information_total_expanded_trips, control_total_expanded_trips]

# Total number of expanded trips per group
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Expanded Trips Per Group')
plt.title('Number of Expanded Trips')

plt.show()

In [ ]:
# Average expanded trips per person
performance = [emotion_total_expanded_trips/emotion_num_users, information_total_expanded_trips/information_num_users, control_total_expanded_trips/control_num_users]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Average Expanded Trips')
plt.title('Average Expanded Trips Per Person')
 
plt.show()

# Permutation Tests

In [ ]:
users = edb.get_uuid_db().find()

In [ ]:
import pandas as pd
from scipy import stats
import emission.storage.timeseries.abstract_timeseries as esta
from datetime import timedelta, date, tzinfo, datetime
import numpy as np

# Create a dataframe with columns user_id, number of diary checks, week number, and group.
df = pd.DataFrame()
information_count = 0
emotion_count = 0
control_count = 0
for i in range(len(users)):
    user_id = users[i]
    start = arrow.get('2018-03-28', 'YYYY-MM-DD')
    end = arrow.get('2018-06-06', 'YYYY-MM-DD')
    vals = []
    week_val = -1
    for week in arrow.Arrow.range('week', start, end):
        ts = esta.TimeSeries.get_time_series(user_id)
        begin_ts = week.timestamp
        end_ts = (week + timedelta(weeks=1)).timestamp
        last_period_tq = estt.TimeQuery("data.start_ts", begin_ts, end_ts)
        cs_df = ts.get_data_df("analysis/inferred_section", time_query=last_period_tq)
        total = 0
        if cs_df.shape[0] <= 0:
            continue
        try:
            for event in expanded_trip_events:
                if event['user_id'] == user_id:
                    if event['ts'] > begin_ts and event['ts'] <= end_ts:
                        total += 1
        except:
            continue
        vals.append(total)
    #Always use lists only where the number of datapoints is greater than 2 otherwise we get a perfect correlation
    weeks = np.arange(len(vals))
    if len(weeks) > 1:
        group = "none"
        try:
            client = edb.get_profile_db().find_one({"user_id": user_id})['client']
            if client == 'urap-2017-information':
                group = "information"
                information_count += 1
            elif client == 'urap-2017-emotion':
                group = "emotion"
                emotion_count += 1
            elif client == 'urap-2017-control':
                group = "control"
                control_count += 1
        except:
            continue
        df = df.append({'uuid': user_id, 'group': group, 'total': sum(vals)}, ignore_index=True)

In [ ]:
df.groupby('group').mean()

In [ ]:
e_c = df[df['group'] != 'information']
sf.perm_test(e_c['group'], e_c['total'], sf.mean_diff, 100000)

In [ ]:
i_c = df[df['group'] != 'emotion']
sf.perm_test(i_c['group'], i_c['total'], sf.mean_diff, 100000)

In [ ]:
i_e = df[df['group'] != 'control']
sf.perm_test(i_e['group'], i_e['total'], sf.mean_diff, 100000)

# Bootstrapping Tests

In [ ]:
e_c = df[df['group'] != 'information']
sf.bootstrap_test(e_c['group'], e_c['total'], sf.mean_diff, 100000)

# Mann Whitney U Tests

In [ ]:
from scipy.stats import mannwhitneyu

control = df[df['group'] == 'control']
control_array = control.as_matrix(columns=control.columns[1:2])

emotion = df[df['group'] == 'emotion']
emotion_array = emotion.as_matrix(columns=emotion.columns[1:2])

print(mannwhitneyu(emotion_array, control_array))